In [2]:
import numpy as np
import pandas as pd
import gzip
import json

from pprint import pprint

In [3]:
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
turkish_stopwords = stopwords.words('turkish')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Osama\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
train_classification_df = pd.read_csv("/Users/Osama/Downloads/CS412PROJ/train-classification.csv",)
train_classification_df = train_classification_df.rename(columns={'Unnamed: 0': 'user_id', 'label': 'category'})

# Unifying labels
train_classification_df["category"] = train_classification_df["category"].apply(str.lower)
username2_category = train_classification_df.set_index("user_id").to_dict()["category"]

In [5]:
train_classification_df.groupby("category").count()

,user_id
category,
art,191
entertainment,323
fashion,299
food,511
gaming,13
health and lifestyle,503
mom and children,149
sports,113
tech,346


In [6]:
username2_category["kod8net"]

'tech'

In [8]:
train_data_path = "/Users/Osama/Downloads/CS412PROJ/training-dataset.jsonl.gz"

username2posts_train = dict()
username2profile_train = dict()

username2posts_test = dict()
username2profile_test = dict()


with gzip.open(train_data_path, "rt") as fh:
  for line in fh:
    sample = json.loads(line)

    profile = sample["profile"]
    username = profile["username"]
    if username in username2_category:
      # train data info
      username2posts_train[username] = sample["posts"]
      username2profile_train[username] = profile


    else:
      # it is test data info
      username2posts_test[username] = sample["posts"]
      username2profile_test[username] = profile

In [9]:
train_profile_df = pd.DataFrame(username2profile_train).T.reset_index(drop=True)
test_profile_df = pd.DataFrame(username2profile_test).T.reset_index(drop=True)

train_profile_df.shape

(2741, 44)

In [48]:
test_profile_df.head(10)

,username,id,full_name,biography,category_name,post_count,follower_count,following_count,is_business_account,is_private,...,business_category_name,overall_category_name,category_enum,is_verified_by_mv4b,is_regulated_c18,profile_pic_url,should_show_category,should_show_public_contacts,show_account_transparency_details,profile_picture_base64
0,beyazyakaliyiz,8634457436,Selam Beyaz Yakalı,Beyaz yakalıların dünyasına hoşgeldiniz 😀😀😀,Personal blog,None,1265,665,True,False,...,None,None,PERSONAL_BLOG,False,False,https://instagram.fist6-1.fna.fbcdn.net/v/t51....,True,True,True,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...
1,totalenergies_istasyonlari,7066643793,TotalEnergies İstasyonları,TotalEnergies İstasyonları resmi Instagram hes...,Energy Company,None,28025,4,True,False,...,None,None,ENERGY_COMPANY,False,False,https://instagram.fsaw2-1.fna.fbcdn.net/v/t51....,True,True,True,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...
2,konforyatak,8782109673,Konfor Yatak #KonforluUykular,"Konfor Yatak, birbirinden farklı özelliklere s...",Furniture,None,40334,2,True,False,...,None,None,FURNITURE,False,False,https://instagram.fyei6-3.fna.fbcdn.net/v/t51....,True,True,True,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...
3,ht_kulup,1950140344,HT KULÜP,Bloomberght - Habertürk Magazin,None,None,158877,69,True,False,...,None,None,TV_SHOW,False,False,https://instagram.fada2-1.fna.fbcdn.net/v/t51....,True,True,True,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...
4,ajansspor,338611487,Ajansspor,"🏢 Saran Group \n🏟 Anında, tarafsız spor haberl...",News & media website,None,93193,286,True,False,...,None,None,NEWS_SITE,False,False,https://instagram.fadb2-1.fna.fbcdn.net/v/t51....,True,True,True,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...
5,yusufelibelediyesi08,43824173432,Yusufeli Belediye Başkanlığı,Yusufeli Belediye Başkanlığı Resmi İnstagram H...,Community,None,4635,83,True,False,...,None,None,None,False,False,https://instagram.fszf2-1.fna.fbcdn.net/v/t51....,False,False,True,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...
6,4bros.tr,8598418995,4 Bros Burger,📍Özlüce/Bursa\nAhmet Taner Kışlalı Bulv.\n📞0 2...,Restaurant,None,5824,5,True,False,...,None,None,RESTAURANT,False,False,https://instagram.fadb3-1.fna.fbcdn.net/v/t51....,True,True,True,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...
7,groundy.kadikoy,5357204834,Groundy Kadıköy,📲 0530 434 3535\nGroundy Kadıköy Güncel Etkinl...,Dance & Night Club,None,14874,17,True,False,...,None,None,NIGHT_CLUB,False,False,https://instagram.fadb2-2.fna.fbcdn.net/v/t51....,True,True,True,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...
8,drtubagunebak,1118780766,Dr. Ç.Tuba Günebak,👩🏼‍⚕️Ankara ofis:Beytepe\n👩🏼‍💼İstanbul ofis:Ba...,Nutritionist,None,44367,3600,True,False,...,None,None,NUTRITIONIST,False,False,https://instagram.fesb4-1.fna.fbcdn.net/v/t51....,True,True,True,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...
9,nihatcan11,7728929668,Nihat Can,None,Politician,None,2749,254,True,False,...,None,None,POLITICIAN,False,False,https://instagram.fesb7-1.fna.fbcdn.net/v/t51....,True,True,True,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...


In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import emoji
from sklearn.feature_selection import SelectKBest, chi2

def preprocess_text(text: str):
    

    # lower casing Turkish Text, Don't use str.lower :)
    text = text.casefold()

    #text = emoji.demojize(text, delimiters=(" ", " "))

    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)

    # Remove special characters and punctuation
    # HERE THE EMOJIS stuff are being removed, you may want to keep them :D
    text = re.sub(r'[^a-zçğıöşü0-9\s#@]', '', text)

    # Remove numbers
    text = re.sub(r'\d+', '', text)

    # Remove extra whitespaces
    text = re.sub(r'\s+', ' ', text).strip()

    return text


corpus = []

# to keep the label order
train_usernames = []

for username, posts in username2posts_train.items():
  train_usernames.append(username)

  # aggregating the posts per user
  cleaned_captions = []
  for post in posts:
    post_caption = post.get("caption", "")
    if post_caption is None:
      continue

    post_caption = preprocess_text(post_caption)

    if post_caption != "":
      cleaned_captions.append(post_caption)


  # joining the posts of each user with a \n
  user_post_captions = "\n".join(cleaned_captions)
  corpus.append(user_post_captions)



#custom_stopwords = list(set(turkish_stopwords).union({
 #   'the', 'and', 'with', 'for', 'you', 'to', 'of', 'in', 'our', 'your', 'is', 'are','bir'
#}))
vectorizer = TfidfVectorizer(stop_words=turkish_stopwords, max_features=15000,min_df=10,sublinear_tf=True,smooth_idf=False,ngram_range=(1, 3))

# fit the vectorizer
vectorizer.fit(corpus)

# transform the data into vectors
x_post_train = vectorizer.transform(corpus)
y_train = [username2_category.get(uname, "NA") for uname in train_usernames]
feature_names = vectorizer.get_feature_names_out()







# Perform chi-squared feature selection
chi2_selector = SelectKBest(chi2, k=5000)  # Select the top 5000 features
x_post_train_selected = chi2_selector.fit_transform(x_post_train, y_train)

# Get the selected feature names
selected_feature_indices = chi2_selector.get_support(indices=True)
selected_feature_names = np.array(feature_names)[selected_feature_indices]



# Convert the selected features into a DataFrame for analysis
df_tfidf = pd.DataFrame(x_post_train_selected.toarray(), columns=selected_feature_names)

# Show the most frequent selected words
print(df_tfidf.sum().sort_values(ascending=False).head(30))





'''
# Inspect the frequency of each word
df_tfidf = pd.DataFrame(x_post_train.toarray(), columns=feature_names)

# Show the most frequent words (words in many posts)
print(df_tfidf.sum().sort_values(ascending=False).head(30))

'''


test_usernames = []
test_corpus = []
for username, posts in username2posts_test.items():
  test_usernames.append(username)
  # aggregating the posts per user
  cleaned_captions = []
  for post in posts:
    post_caption = post.get("caption", "")
    if post_caption is None:
      continue

    post_caption = preprocess_text(post_caption)

    if post_caption != "":
      cleaned_captions.append(post_caption)

  user_post_captions = "\n".join(cleaned_captions)
  test_corpus.append(user_post_captions)


# Just transforming! No Fitting!!!!!
x_post_test = vectorizer.transform(test_corpus)




# Transform the test set using the same feature selection
x_post_test_selected = chi2_selector.transform(x_post_test)

the              43.234850
olarak           40.352663
kutlu            38.096409
kutlu olsun      37.759918
güzel            37.389047
and              34.504780
of               34.106894
to               33.099203
birlikte         32.901064
in               32.374707
with             28.743966
bilgi            28.325731
teşekkür         28.247761
mustafa          27.676218
for              27.280477
türkiye          27.194764
you              26.948208
kemal            26.702591
is               25.541688
mustafa kemal    25.536023
mi               25.390266
ziyaret          25.182333
olmak            24.740089
ben              24.492370
sonra            24.243549
hafta            24.095780
lezzet           23.966902
bekliyor         23.044388
bekliyoruz       22.340900
our              22.187366
dtype: float64


In [12]:
# Making sure everything is fine
assert y_train.count("NA") == 0


In [13]:
feature_names = vectorizer.get_feature_names_out()
feature_names

array(['ab', 'abant', 'abd', ..., 'şıklığın', 'şımartın', 'şırnak'],
      dtype=object)

In [14]:
df_tfidf = pd.DataFrame(x_post_train.toarray(), columns=feature_names)
df_tfidf.head(2)

,ab,abant,abd,abdulkadir,abdullah,abi,ability,abiye,able,able to,...,şükürler,şükürler olsun,şüphesiz,şık,şık bir,şıklık,şıklığı,şıklığın,şımartın,şırnak
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.04435,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
df_tfidf.shape

(2741, 15000)

In [16]:
print("Shape of df_tfidf:", df_tfidf.shape)
print("Length of y_train:", len(y_train))

Shape of df_tfidf: (2741, 15000)
Length of y_train: 2741


In [17]:
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(df_tfidf, y_train, test_size=0.2, stratify=y_train,random_state=42)

In [18]:
x_train.shape

(2192, 15000)

In [19]:
x_val.shape

(549, 15000)

In [20]:
#@title Test Data
test_data_path = "/Users/Osama/Downloads/CS412PROJ/test-classification-round1.dat"

with open(test_data_path, "rt") as fh:
    for i, line in enumerate(fh):
        print(line.strip())
        if i >= 4:  # Stop after 5 lines
            break

print("*****")

test_unames = []
with open(test_data_path, "rt") as fh:
  for line in fh:
    test_unames.append(line.strip())

print(test_unames[:5])

ozhotelstr
elleturkiye
sozerinsaatorhangazi
sanliurfapiazzaavym
rusanozden
*****
['ozhotelstr', 'elleturkiye', 'sozerinsaatorhangazi', 'sanliurfapiazzaavym', 'rusanozden']


In [21]:
x_test = []

for uname in test_unames:
  try:
    index = test_usernames.index(uname)
    x_test.append(x_post_test[index].toarray()[0])
  except Exception as e:
    try:
      index = train_usernames.index(uname)
      x_test.append(x_post_train[index].toarray()[0])
    except Exception as e:
      print(uname)


test_unames.remove("screenname")

screenname


In [22]:
df_test = pd.DataFrame(np.array(x_test), columns=feature_names)
df_test.head(2)

,ab,abant,abd,abdulkadir,abdullah,abi,ability,abiye,able,able to,...,şükürler,şükürler olsun,şüphesiz,şık,şık bir,şıklık,şıklığı,şıklığın,şımartın,şırnak
0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.027864,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.031456,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
def predict_like_count(username, current_post=None):
  def get_avg_like_count(posts:list):
    total = 0.
    for post in posts:
      if current_post is not None and post["id"] == current_post["id"]:
        continue

      like_count = post.get("like_count", 0)
      if like_count is None:
        like_count = 0
      total += like_count

    if len(posts) == 0:
      return 0.

    return total / len(posts)

  if username in username2posts_train:
    return get_avg_like_count(username2posts_train[username])
  elif username in username2posts_test:
    return get_avg_like_count(username2posts_test[username])
  else:
    print(f"No data available for {username}")
    return -1

In [24]:
def log_mse_like_counts(y_true, y_pred):
  """
  Calculate the Log Mean Squared Error (Log MSE) for like counts (log(like_count + 1)).

  Parameters:
  - y_true: array-like, actual like counts
  - y_pred: array-like, predicted like counts

  Returns:
  - log_mse: float, Log Mean Squared Error
  """
  # Ensure inputs are numpy arrays
  y_true = np.array(y_true)
  y_pred = np.array(y_pred)

  # Log transformation: log(like_count + 1)
  log_y_true = np.log1p(y_true)
  log_y_pred = np.log1p(y_pred)

  # Compute squared errors
  squared_errors = (log_y_true - log_y_pred) ** 2

  # Return the mean of squared errors
  return np.mean(squared_errors)

In [25]:
#@title Train Dataset evaluation

y_like_count_train_true = []
y_like_count_train_pred = []
for uname, posts in username2posts_train.items():
  for post in posts:
    pred_val = predict_like_count(uname, post)
    true_val = post.get("like_count", 0)
    if true_val is None:
      true_val = 0

    y_like_count_train_true.append(true_val)
    y_like_count_train_pred.append(pred_val)

print(f"Log MSE Train= {log_mse_like_counts(y_like_count_train_true, y_like_count_train_pred)}")

Log MSE Train= 1.2271047744059362


In [26]:
#@title Test Dataset

path = "/Users/Osama/Downloads/CS412PROJ/test-regression-round1.jsonl"
output_path = "/Users/Osama/Downloads/CS412PROJ/testy-regression-round1.jsonl"

to_predict_like_counts_usernames = []
output_list = []
with open(path, "rt") as fh:
  for line in fh:
    sample = json.loads(line)

    # let's predict
    pred_val = predict_like_count(sample["username"])
    sample["like_count"] = int(pred_val)
    output_list.append(sample)

with open(output_path, "wt") as of:
  json.dump(output_list, of)




In [27]:
pprint(output_list[:3])

[{'caption': 'KOZA 2023 2.si Damla’nın koleksiyonu, Latincede ‘Memento Mori’ '
             'olarak bilinen ‘ölümlü olduğunu hatırla’ anlamındaki ifadeden '
             'esinleniyor. Koleksiyon, hayatın ve ölümün, para, işçi, kral ve '
             'kraliçe kavramları üzerinden yaratıcı görünümlerle bir araya '
             'getirilmesini amaçlıyor. Ölüm sembollerinden esinlenen desenler '
             'kullanan Damla, “kağıt parçasından ibaret olmak” kavramını '
             'vurguluyor. Koleksiyon, yaşamın ve ölümün aynı anda ifade '
             'edilmesini hedefliyor; kırmızı ve mavi ışıklarla veya '
             'gözlüklerle görülen hologram efekti kullanılarak bu konsept '
             'sahneye taşınıyor. Kırmızı renk ölümü, mavi ise yaşamı '
             'simgeliyor. Koleksiyon, ofis giyimlerinden esinlenerek '
             'kravatlar, gömlekler ve evrak çantaları içeriyor. Klasik sivri '
             'burun çizmelerin üzerine spor ayakkabıların üst yüzeyi '
             'yerle

In [28]:
'''from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor


# Define the parameter grid
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2']
}

# Initialize the regressor
rf = RandomForestRegressor(random_state=42)

# Perform grid search
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', verbose=2, n_jobs=-1)
grid_search.fit(x_train, y_train)

# Best parameters and best model
print("Best Parameters:", grid_search.best_params_)
best_model = grid_search.best_estimator_
'''

'from sklearn.model_selection import GridSearchCV\nfrom sklearn.ensemble import RandomForestRegressor\n\n\n# Define the parameter grid\nparam_grid = {\n    \'n_estimators\': [50, 100, 200],\n    \'max_depth\': [None, 10, 20, 30],\n    \'min_samples_split\': [2, 5, 10],\n    \'min_samples_leaf\': [1, 2, 4],\n    \'max_features\': [\'auto\', \'sqrt\', \'log2\']\n}\n\n# Initialize the regressor\nrf = RandomForestRegressor(random_state=42)\n\n# Perform grid search\ngrid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, scoring=\'neg_mean_squared_error\', verbose=2, n_jobs=-1)\ngrid_search.fit(x_train, y_train)\n\n# Best parameters and best model\nprint("Best Parameters:", grid_search.best_params_)\nbest_model = grid_search.best_estimator_\n'

In [29]:
from xgboost import XGBRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

# Prepare the target variable: Average like count per user
user_avg_likes = {}
for username, posts in username2posts_train.items():
    avg_likes = np.mean([post.get("like_count", 0) or 0 for post in posts])
    user_avg_likes[username] = avg_likes

# Create the target array
y_like_counts = [user_avg_likes[uname] for uname in train_usernames]
y_like_counts = [0 if np.isnan(val) else val for val in y_like_counts]

# Train-test split
x_train, x_val, y_train, y_val = train_test_split(
    x_post_train, y_like_counts, test_size=0.2, random_state=42
)

# Train an MLP Regressor
xgb_regressor = XGBRegressor(
    n_estimators=100,   # Number of trees
    learning_rate=0.1,  # Step size for updating weights
    max_depth=6,        # Depth of each tree
    random_state=42
)

# Train the model
xgb_regressor.fit(x_train, y_train)

y_val = np.maximum(y_val, 0)



# Predict on validation set
y_pred_val = xgb_regressor.predict(x_val)
y_pred_val = np.maximum(y_pred_val, 0)

# Evaluate the model
mse = mean_squared_error(y_val, y_pred_val)
print(f"MSE on validation set: {mse}")

# Predict on test set
y_test_pred = xgb_regressor.predict(x_post_test)


# Create a DataFrame for test predictions
test_predictions = pd.DataFrame({
    "username": test_usernames,
    "predicted_like_count": y_test_pred
})
print(test_predictions.head())


C:\Users\Osama\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\Osama\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


MSE on validation set: 2844503843.9501276
                     username  predicted_like_count
0              beyazyakaliyiz           6031.495117
1  totalenergies_istasyonlari            333.563660
2                 konforyatak            333.563660
3                    ht_kulup            318.940491
4                   ajansspor            961.202271


In [44]:
import statsmodels.api as sm
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

# Prepare the target variable: Average like count per user
user_avg_likes = {}
for username, posts in username2posts_train.items():
    avg_likes = np.mean([post.get("like_count", 0) or 0 for post in posts])
    user_avg_likes[username] = avg_likes

# Create the target array
y_like_counts = [user_avg_likes[uname] for uname in train_usernames]
y_like_counts = [0 if np.isnan(val) else val for val in y_like_counts]

# Train-test split
x_train, x_val, y_train, y_val = train_test_split(
    x_post_train, y_like_counts, test_size=0.2, random_state=42
)

lgbm_regressor = LGBMRegressor(
    n_estimators=1000,
    learning_rate=0.2,
    max_depth=30,
    min_data_in_leaf=30,
    feature_fraction=0.8,
    bagging_fraction=0.8,
    objective="poisson",  # Poisson objective for count data
    random_state=42
)

# Train the model
lgbm_regressor.fit(x_train, y_train)

# Predict on validation set
y_pred_val = lgbm_regressor.predict(x_val)

# Ensure non-negative predictions
y_pred_val = np.maximum(y_pred_val, 0)

# Evaluate the model
mse = log_mse_like_counts(y_val, y_pred_val)
print(f"MSE on validation set: {mse}")

# Predict on test set
y_test_pred = lgbm_regressor.predict(x_post_test)

# Ensure non-negative predictions
y_test_pred = np.maximum(y_test_pred, 0)

# Create a DataFrame for test predictions
test_predictions = pd.DataFrame({
    "username": test_usernames,
    "predicted_like_count": y_test_pred
})

print(test_predictions.head())

C:\Users\Osama\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\Osama\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=30
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=30
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.090126 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 212264
[LightGBM] [Info] Number of data points in the train set: 2192, number of us

In [37]:
print(test_predictions.head(50))

                      username  predicted_like_count
0               beyazyakaliyiz           6558.287578
1   totalenergies_istasyonlari             20.860497
2                  konforyatak             22.142566
3                     ht_kulup            188.599836
4                    ajansspor            762.109647
5         yusufelibelediyesi08            143.722016
6                     4bros.tr             92.746365
7              groundy.kadikoy            605.133060
8                drtubagunebak            136.641764
9                   nihatcan11            176.347690
10               mustafaaakcay            282.351146
11                  mtmofamily           7699.112320
12                 bonitoperde            152.304595
13     balkanlardangelenlezzet             33.321607
14      pedagoganne__gulozturk            459.071655
15                      iktm34            126.996710
16                    yudumyag            361.955106
17      bulentozdemir.edebiyat            427.

In [38]:
if np.isnan(y_val).any() or np.isnan(y_pred_val).any():
    print("NaN values found in y_val or y_pred_val!")


In [45]:
log_errors = np.log1p(y_val) - np.log1p(y_pred_val)  # Use np.log1p to handle zero values gracefully

# Compute the absolute log errors
abs_log_errors = np.abs(log_errors)

# Sum of log errors
sum_log_errors = np.sum(abs_log_errors)

print(f"Sum of Log Errors: {sum_log_errors}")

Sum of Log Errors: 727.0928358570148


In [46]:
print(f"Log MSE Test= {log_mse_like_counts(y_val, y_pred_val)}")

Log MSE Test= 2.975259638495896


In [140]:
'''param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.05, 0.1, 0.15],
    'max_depth': [5, 10, -1],
    'min_data_in_leaf': [10, 20, 30],
    'feature_fraction': [0.8, 0.9],
    'bagging_fraction': [0.8, 0.9]
}

from sklearn.model_selection import GridSearchCV

grid_search = GridSearchCV(estimator=lgbm_regressor, param_grid=param_grid, cv=3, n_jobs=-1, scoring='neg_mean_squared_error')
grid_search.fit(x_train, y_train)

print("Best parameters found: ", grid_search.best_params_)'''


KeyboardInterrupt: 

In [78]:
'''from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

# Prepare the target variable: Average like count per user
user_avg_likes = {}
for username, posts in username2posts_train.items():
    avg_likes = np.mean([post.get("like_count", 0) or 0 for post in posts])
    user_avg_likes[username] = avg_likes

# Create the target array
y_like_counts = [user_avg_likes[uname] for uname in train_usernames]
y_like_counts = [0 if np.isnan(val) else val for val in y_like_counts]
# Train-test split
x_train, x_val, y_train, y_val = train_test_split(
    x_post_train, y_like_counts, test_size=0.2, random_state=42
)

xgb_regressor = XGBRegressor(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=6,
    objective='reg:squarederror',  # Default; can change to 'reg:poisson' for count data
    random_state=42
)

# Train the model
xgb_regressor.fit(x_train, y_train)

# Predict on validation set
y_pred_val = xgb_regressor.predict(x_val)

# Post-process: Clamp negative predictions to 0
y_pred_val = np.maximum(y_pred_val, 0)

# Evaluate the model
mse = mean_squared_error(y_val, y_pred_val)
log_errors = np.log1p(y_val) - np.log1p(y_pred_val)  # Use np.log1p to handle zero values
abs_log_errors = np.abs(log_errors)
sum_log_errors = np.sum(abs_log_errors)

print(f"MSE on validation set: {mse}")
print(f"Sum of Log Errors: {sum_log_errors}")

# Predict on test set
y_test_pred = xgb_regressor.predict(x_post_test)

# Post-process: Clamp negative predictions to 0
y_test_pred = np.maximum(y_test_pred, 0)

# Create a DataFrame for test predictions
test_predictions = pd.DataFrame({
    "username": test_usernames,
    "predicted_like_count": y_test_pred
})

print(test_predictions.head())

# Save test predictions to a CSV file
#test_predictions.to_csv("test_predictions.csv", index=False)'''


C:\Users\Osama\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\numpy\_core\fromnumeric.py:3596: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\Osama\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\numpy\_core\_methods.py:138: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


MSE on validation set: 2819676604.812782
Sum of Log Errors: 1278.0898952402472
                     username  predicted_like_count
0              beyazyakaliyiz          10770.258789
1  totalenergies_istasyonlari            453.646881
2                 konforyatak            370.162567
3                    ht_kulup            484.007294
4                   ajansspor           2237.495117


In [63]:
# Check for NaN in user_avg_likes
for username, posts in username2posts_train.items():
    avg_likes = np.mean([post.get("like_count", 0) or 0 for post in posts])
    if np.isnan(avg_likes):
        print(f"NaN detected for user: {username}, posts: {posts}")
    user_avg_likes[username] = avg_likes


user_avg_likes["touchdownistanbul"]

NaN detected for user: touchdownistanbul, posts: []
NaN detected for user: belediyesikose, posts: []


C:\Users\Osama\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\Osama\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


nan

In [ ]:
import json
import numpy as np
import pandas as pd

# Path to the test file
test_data_path = "/Users/Osama/Downloads/CS412PROJ/test-regression-round3.jsonl"
test_data = []
with open(test_data_path, "r") as file:
    for line in file:
        test_data.append(json.loads(line.strip()))



# Extract user IDs and captions
test_user_ids = [entry["id"] for entry in test_data]
test_unames = [entry["username"] for entry in test_data]
x_test = []

for uname in test_unames:
  try:
    index = test_usernames.index(uname)
    x_test.append(x_post_test[index].toarray()[0])
  except Exception as e:
    try:
      index = train_usernames.index(uname)
      x_test.append(x_post_train[index].toarray()[0])
    except Exception as e:
      print(uname)

'''
for id in test_user_ids:
  try:
    index = test_user_ids.index(id)
    x_test.append(x_post_test_selected[index].toarray()[0])
  except Exception as e:
    try:
      index = train_usernames.index(id)
      x_test.append(x_post_train_selected[index].toarray()[0])
    except Exception as e:
      print(id)
'''

#test_captions = [entry.get("caption", "") for entry in test_data]

# Preprocess test captions
#preprocessed_captions = [preprocess_text(caption) for caption in test_captions]

# Transform test captions using trained vectorizer and selector
#x_post_test_transformed = vectorizer.transform(preprocessed_captions)
#x_post_test_selected = chi2_selector.transform(x_post_test_transformed)

# Make predictions
y_test_predictions = lgbm_regressor.predict(x_test)

# Ensure non-negative predictions
y_test_predictions = np.maximum(y_test_predictions, 0)
# Prepare the output dictionary
output = {user_id: int(prediction) for user_id, prediction in zip(test_user_ids, y_test_predictions.tolist())}

#output = {user_id: int(prediction) for user_id, prediction in zip(test_user_ids, y_test_predictions)}

# Save predictions to a JSON file
output_file_path = "output_predictions_reg.json"
with open(output_file_path, "w") as output_file:
    json.dump(output, output_file, indent=4)

print(f"Predictions saved to {output_file_path}")


[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=30
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
Predictions saved to output_predictions_reg.json


In [55]:
print(test_data[0])  # Print the first entry to inspect its structure


{'caption': 'Ayvalık Belediyesi Kasım Ayı Olağan Meclis Toplantısı’nı Vural Sineması Nejat Uygur Sahnesi’nde gerçekleştirdik.\n\nAlınan kararların Ayvalık’ımıza hayırlı olmasını dilerim.', 'comments_count': 10, 'id': '18299464882193238', 'media_type': 'CAROUSEL_ALBUM', 'media_url': 'https://scontent-sof1-2.cdninstagram.com/v/t51.29350-15/398502640_367498438952512_6710374471736346193_n.jpg?_nc_cat=107&ccb=1-7&_nc_sid=c4dd86&_nc_ohc=QFdBTP4J-X8AX8jZhY4&_nc_ht=scontent-sof1-2.cdninstagram.com&edm=AL-3X8kEAAAA&oh=00_AfDe8viPR0XWIPWwP_ravKlJ1b7kqr_pxMs0DT7IIf6DIQ&oe=65585C5E', 'timestamp': '2023-11-01 12:43:50', 'username': 'mesuterginofficial'}
